In [1]:
import requests
from bs4 import BeautifulSoup


Picked 'user-agent' for my browser from https://www.whatismybrowser.com/detect/what-is-my-user-agent/



In [2]:
url = "https://patient.info/forums/discuss/browse/depression-683"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    #Switch the User-agent to avoid anti-scraping measures and "make believe" this is a real human requests those URLs
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

print(soup)


<!DOCTYPE html>

<html data-js="no" lang="en">
<head data-app-version="6.2.5" data-userid="">
<meta charset="utf-8"/>
<script>document.documentElement.setAttribute('data-js', 'yes');</script>
<title>Depression | Mental health | Forums | Patient</title>
<link href="https://patient.azureedge.net/gfx/icons/apple-touch-icon-pipa.png" rel="preload apple-touch-icon" sizes="180x180"/>
<link href="https://patient.azureedge.net/gfx/icons/favicon-32x32-pipa.png" rel="preload icon" sizes="32x32" type="image/png"/>
<link href="https://patient.azureedge.net/gfx/icons/favicon-16x16-pipa.png" rel="preload icon" sizes="16x16" type="image/png"/>
<link href="https://patient.azureedge.net/gfx/icons/manifest.json" rel="preload manifest"/>
<link color="#5bbad5" href="https://patient.azureedge.net/gfx/icons/safari-pinned-tab-pipa.svg" rel="preload mask-icon"/>
<link href="https://patient.azureedge.net/gfx/icons/favicon-pipa.ico" rel="preload shortcut icon"/>
<link href="/manifest.json" rel="manifest"/>
<me

In [5]:
from lxml import etree

In [26]:
dom = etree.HTML(str(soup))

for j in range(1, 37):
    try:
        print('https://patient.info'+dom.xpath(f'/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/div/ul/li[{j}]/div/div[2]/article/h3/a/@href')[0])
    except:
        continue

https://patient.info/forums/discuss/parox-791813
https://patient.info/forums/discuss/depression-or-something-more-complex--791524
https://patient.info/forums/discuss/effexor-withdrawal-how-long-will-these-feelings-last--488147
https://patient.info/forums/discuss/asking-for-the-other-half--791541
https://patient.info/forums/discuss/rapid-weight-gain-after-stopping-venlafaxine-564278
https://patient.info/forums/discuss/ashamed-in-40s-790546
https://patient.info/forums/discuss/coming-off-pristiq-and-amitriptyline-at-the-same-time-790968
https://patient.info/forums/discuss/i-have-two-sides-790905
https://patient.info/forums/discuss/suicide-attempt-missed-790609
https://patient.info/forums/discuss/cut-off-left-adrift-feel-dejected-790638
https://patient.info/forums/discuss/i-wish-i-was-dead-tired-of-living--499345
https://patient.info/forums/discuss/severely-depressed-no-drive-anymore-784720
https://patient.info/forums/discuss/always-negative-thoughts-and-hypochondria-787936
https://patient

In [27]:
####        PSEUDO-CODE       #####
### LOOP THROUGH EACH PAGE FROM 1 TO 198 starting with : 
### https://patient.info/forums/discuss/browse/depression-683?page=0#group-discussions LOOP AROUND PAGE = 0 
### then PAGE = 1 then PAGE = 2...up to 197

post_urls = []

for i in range(198):
    page_url = f"https://patient.info/forums/discuss/browse/depression-683?page={i}#group-discussions"
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for j in range(1, 37):
        XPATH = f"/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/div/ul/li[{j}]/div/div[2]/article/h3/a/@href"
        
        dom = etree.HTML(str(soup))
        
        try:
            post_url = 'https://patient.info' + dom.xpath(XPATH)[0]
        except:
            continue
        
        post_urls.append(post_url)

### FOR EACH TOPIC NAME (use XPATH) GRAB URL AND APPEND TO A LIST

### THEN LOOP THROUGH EVERY URL IN LIST, REQUEST URL, AND SCRAP MAIN POST'S TEXT

len(post_urls)

6534

In [32]:
post_urls[:5]

['https://patient.info/forums/discuss/parox-791813',
 'https://patient.info/forums/discuss/depression-or-something-more-complex--791524',
 'https://patient.info/forums/discuss/effexor-withdrawal-how-long-will-these-feelings-last--488147',
 'https://patient.info/forums/discuss/asking-for-the-other-half--791541',
 'https://patient.info/forums/discuss/rapid-weight-gain-after-stopping-venlafaxine-564278']

In [34]:
with open('./post_urls.txt', 'w') as f:
    f.write('\n'.join(post_urls))
